In [1]:
import nltk
import string
import spacy
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import accuracy_score
from spacy.lang.en import English
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = PorterStemmer( )
lemmatizer = WordNetLemmatizer( )

def clean_text_and_get_tokens( text ):
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    
    return words



# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    sentence = sentence.lower()
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    

    # return preprocessed list of tokens
    return mytokens

In [2]:
def stem_and_lemmatize( sentence ):
    
    tokens = spacy_tokenizer( sentence ) 
    final_tokens = set( )
    for token in tokens:
        token = stemmer.stem( token )
        token = lemmatizer.lemmatize( token )
        final_tokens.add( token )
    return list( final_tokens )
    
def update_word_count( x ):
    
    x['word_counts' ]['id' ] = x['id' ]
    x['word_counts' ]['target' ] = x['target']
    return x['word_counts'] 

In [3]:

def get_meaningful_cols( cols ):
    
    meaningful_cols = [ ]
    for col in cols:
        col = str( col )
        if len( col ) <= 2 :
            continue
        cur_sequence_len = 0
        prev_character = col[ 0 ]
        idx = 0
        
        for character in col:
            if character == prev_character:
                cur_sequence_len += 1
            else:
                cur_sequence_len = 1
            prev_character = character
            
            if cur_sequence_len>=3:
                break
                
            idx += 1
            
        if len( col ) == idx :
            meaningful_cols.append( col )
    
        
    return meaningful_cols 
            

In [4]:
def get_tf_idf_df( df , corpus_col ):
    
    vectorizer = TfidfVectorizer( )
    
    X = vectorizer.fit_transform( df[ corpus_col ].tolist( ) )
    
    new_df = pd.DataFrame( X.toarray() , columns = vectorizer.get_feature_names() )
    
    new_df[ 'id' ] = df[ 'id' ] 
    
    return new_df

In [5]:
train_df = pd.read_csv( 'train.csv')

train_df[ 'word_tokens' ] = train_df[ 'text' ].apply(clean_text_and_get_tokens).apply(lambda x : stem_and_lemmatize( ' '.join( x ) ) )

pd.set_option('display.max_columns', 500)

# train_df[ 'word_counts' ] = train_df[ 'word_tokens' ].apply( lambda x : Counter( x ) )

# train_df[ 'word_counts' ] = train_df.apply( update_word_count , axis = 1 )

# new_train_df = pd.DataFrame( data = train_df[ 'word_counts' ].tolist() )
train_df[ 'corpus_text'  ] = train_df[ 'word_tokens' ] .apply( lambda x : ' '.join( x  ) )

new_train_df = get_tf_idf_df( train_df , 'corpus_text' )

In [6]:
new_train_df.head()

,aa,aaaa,aaaaaaallll,aaaaaand,aaarrrgghhh,aaceorg,aan,aannnnd,aar,aaronthefm,aashiqui,ab,aba,abandon,abandonedp,abbandon,abbott,abbruchsimul,abbswinston,abbyairshow,abc,abcnew,abcnorio,abe,aberdeen,aberdeenfanpag,aberdeenfc,aberystwythshrewsburi,abha,abia,abil,abject,abl,ablaz,ableg,abninfvet,aboard,abomb,abomin,abort,about,abrancaballero,absenc,absolut,abstorm,abstract,absurd,absurdli,abus,abuseddesol,abysmaljoin,ac,acaciapenn,academia,acarewornheart,acc,accept,access,accid,accident,accidentalpropheci,accionempresa,accompani,accord,accordingli,account,accuraci,accus,accustom,acdelco,ace,acebab,acebreakingnew,acenewsdesk,ach,achedin,achiev,achimota,acid,acmilan,acn,acoust,acousticmaloley,acquiesc,acquir,acquisit,acr,acronym,acryl,act,actavi,actin,action,actionmoviestaughtu,activ,activis,activist,actor,actress,actual,acura,acut,ad,adam,adamantli,adamniblo,adamrubinespn,adamtuss,adani,adann,adapt,add,addict,addit,address,adelaid,adhd,adida,adiossuperbacteria,adjust,admin,administr,admit,adndotcom,adopt,ador,adorableapppl,adrianpeel,adriasimon,adriennetomah,adsit,adult,adultblackmal,adumbbb,advanc,advantag,adventur,advers,advertis,advic,advis,advisori,adweek,aeg,aelinrhe,aeroplan,aerospac,aesop,aesthet,af,afc,affair,affect,affili,affleck,afflict,afghan,afghanistan,afghetcleft,afloat,afp,afraid,africa,african,africanbaz,africansinsf,afrikaan,afrin,aft,afterhaiyan,afterhour,afterlif,aftermath,afternoon,aftershock,aftershockdelo,aftershockorg,afycso,ag,agalloch,agdq,age,agenc,agent,aggarw,aggress,aggressif,agnivesh,agnu,ago,agochicago,agoni,agre,agreement,agreesh,aguero,agusa,agw,ah,ahahahga,ahamedi,ahead,ahh,ahhhh,ahhhhh,ahhtheenikki,ahmaz,ahrar,ahuh,ai,aia,aid,aidad,aidan,aiginsur,aiiamericangiri,aiii,aim,aimlessli,aintsheperti,air,airasia,aircraft,airhead,airhorn,airlift,airlin,airplan,airplaneåê,airport,airstrik,airwav,aisl,aisumag,aitchkayce,ajabrown,ajw,ak,aka,akam,akcsl,akgovbillwalk,akilah,akito,akram,akumareisu,akwa,akx,akxbskdn,al,...,wqow,wr,wraith,wrap,wrapup,wreak,wreck,wreckag,wrestleon,wrestler,wrightsboro,wrinkl,wrist,wristband,write,writebothfist,writer,writingtip,written,wrked,wron,wrong,wrongdejavu,wrongperson,wrongway,wrote,wroug,wrought,wsazbrittani,wsj,wsjthinktank,wsl,wsoar,wsoc,wtc,wtf,wth,wthat,wtoni,wtwitter,wud,wugli,wunscreen,wut,ww,wwa,wwe,wwexdream,wwi,wwii,wwp,www,wwwbigbaldhead,wwwcbplawyer,wwwinformationngcom,wx,wxiatv,wxii,wxki,wy,wyou,wyrmwood,wzbt,xavier,xaviermarqui,xbox,xboxon,xd,xdescri,xdojjjj,xekstrin,xela,xeni,xfile,xgninfin,xhnew,xii,xkdrx,xl,xma,xmen,xo,xoxo,xp,xpost,xray,xv,xvii,xxhjesc,xxx,xylodemon,ya,yaboiluk,yagitudeh,yahistor,yahoo,yahoocar,yahoofin,yahoonew,yahoonewsdigest,yahooschwab,yahootv,yakub,yamaguchi,yamashiro,yanke,yard,yay,yazidi,yazidishingalgenocid,ybtheprophet,yday,ye,yea,yeaahh,yeah,year,yeat,yeda,yeehaw,yehuda,yell,yelllowheath,yellow,yellowston,yelp,yemen,yemeni,yennora,yep,yeshayad,yessum,yesterday,yeyeulala,yh,yhngsjlg,yiayplan,yield,yike,yiraneuni,ykelquiban,ymcglaun,yo,yobe,yoeni,yoga,yogurt,yolandaph,yolk,yonew,yor,york,yorker,yorkshir,yosemit,yougov,young,younger,youngheroesid,youngin,youngsaf,yourboyshawn,youssefyamani,youth,youtu,youtub,youuu,ypg,ypre,yr,yuan,yug,yugvani,yuki,yum,yumiko,yup,yuppi,yuuko,yuvi,yyc,yycfring,yycstorm,yycwalk,yycweath,yyeso,yyj,yzf,zaatari,zabadani,zacb,zach,zachlowenba,zachzaidman,zaibatsunew,zakbagan,zakuun,zaman,zamtriossu,zar,zarharzar,zarri,zayn,zaynmaiikist,zaynmalik,zeal,zehr,zenandemcfen,zenit,zergel,zero,zhejiang,zhenghxn,zicac,zimbabw,zimmer,zimmerman,zimpapersview,zionism,zionist,zip,ziphimup,zippednew,zipper,zippolin,ziuw,zmne,zodiac,zojadelin,zomatoau,zombi,zone,zonesthank,zoom,zotar,zouma,zourryart,zrnf,zumiez,zurich,zxatheti,zzzz,åç,åè,åê,åêfedex,åêi,ìñ,ìü,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [7]:
input_cols = new_train_df.columns.difference( [ 'id' , 'target' ])
input_cols = get_meaningful_cols( input_cols )

op_cols    = ['target' ]

input_df   = new_train_df[ input_cols ] 

op_df      = train_df[ op_cols ]

In [8]:
input_df.head()

,aaceorg,aan,aar,aaronthefm,aashiqui,aba,abandon,abandonedp,abbandon,abbott,abbruchsimul,abbswinston,abbyairshow,abc,abcnew,abcnorio,abe,aberdeen,aberdeenfanpag,aberdeenfc,aberystwythshrewsburi,abha,abia,abil,abject,abl,ablaz,ableg,abninfvet,aboard,abomb,abomin,abort,about,abrancaballero,absenc,absolut,abstorm,abstract,absurd,absurdli,abus,abuseddesol,abysmaljoin,acaciapenn,academia,acarewornheart,acc,accept,access,accid,accident,accidentalpropheci,accionempresa,accompani,accord,accordingli,account,accuraci,accus,accustom,acdelco,ace,acebab,acebreakingnew,acenewsdesk,ach,achedin,achiev,achimota,acid,acmilan,acn,acoust,acousticmaloley,acquiesc,acquir,acquisit,acr,acronym,acryl,act,actavi,actin,action,actionmoviestaughtu,activ,activis,activist,actor,actress,actual,acura,acut,adam,adamantli,adamniblo,adamrubinespn,adamtuss,adani,adann,adapt,add,addict,addit,address,adelaid,adhd,adida,adiossuperbacteria,adjust,admin,administr,admit,adndotcom,adopt,ador,adrianpeel,adriasimon,adriennetomah,adsit,adult,adultblackmal,advanc,advantag,adventur,advers,advertis,advic,advis,advisori,adweek,aeg,aelinrhe,aeroplan,aerospac,aesop,aesthet,afc,affair,affect,affili,affleck,afflict,afghan,afghanistan,afghetcleft,afloat,afp,afraid,africa,african,africanbaz,africansinsf,afrikaan,afrin,aft,afterhaiyan,afterhour,afterlif,aftermath,afternoon,aftershock,aftershockdelo,aftershockorg,afycso,agalloch,agdq,age,agenc,agent,aggarw,aggress,aggressif,agnivesh,agnu,ago,agochicago,agoni,agre,agreement,agreesh,aguero,agusa,agw,ahahahga,ahamedi,ahead,ahh,ahhtheenikki,ahmaz,ahrar,ahuh,aia,aid,aidad,aidan,aiginsur,aiiamericangiri,aim,aimlessli,aintsheperti,air,airasia,aircraft,airhead,airhorn,airlift,airlin,airplan,airplaneåê,airport,airstrik,airwav,aisl,aisumag,aitchkayce,ajabrown,ajw,aka,akam,akcsl,akgovbillwalk,akilah,akito,akram,akumareisu,akwa,akx,akxbskdn,ala,alabama,alabamaquak,aladdin,alameda,alamodc,alanhahn,alarm,alarmem,alarmingli,alaska,alaskan,alaskaseafood,alba,albani,albeit,alberta,albertan,albertson,album,...,worldoil,worldpay,worldvis,worldwatchesferguson,worldwid,worm,worn,worri,wors,worseit,worsen,worship,worst,worstev,worstoverdos,worstsummerjob,worth,worthless,wouldelectrocut,wound,woundedpigeon,wout,wow,wowo,wowsavannah,wowth,wpd,wpo,wpri,wqow,wraith,wrap,wrapup,wreak,wreck,wreckag,wrestleon,wrestler,wrightsboro,wrinkl,wrist,wristband,write,writebothfist,writer,writingtip,written,wrked,wron,wrong,wrongdejavu,wrongperson,wrongway,wrote,wroug,wrought,wsazbrittani,wsj,wsjthinktank,wsl,wsoar,wsoc,wtc,wtf,wth,wthat,wtoni,wtwitter,wud,wugli,wunscreen,wut,wwa,wwe,wwexdream,wwi,wwii,wwp,wxiatv,wxii,wxki,wyou,wyrmwood,wzbt,xavier,xaviermarqui,xbox,xboxon,xdescri,xekstrin,xela,xeni,xfile,xgninfin,xhnew,xii,xkdrx,xma,xmen,xoxo,xpost,xray,xvii,xxhjesc,xylodemon,yaboiluk,yagitudeh,yahistor,yahoo,yahoocar,yahoofin,yahoonew,yahoonewsdigest,yahooschwab,yahootv,yakub,yamaguchi,yamashiro,yanke,yard,yay,yazidi,yazidishingalgenocid,ybtheprophet,yday,yea,yeaahh,yeah,year,yeat,yeda,yeehaw,yehuda,yell,yellow,yellowston,yelp,yemen,yemeni,yennora,yep,yeshayad,yessum,yesterday,yeyeulala,yhngsjlg,yiayplan,yield,yike,yiraneuni,ykelquiban,ymcglaun,yobe,yoeni,yoga,yogurt,yolandaph,yolk,yonew,yor,york,yorker,yorkshir,yosemit,yougov,young,younger,youngheroesid,youngin,youngsaf,yourboyshawn,youssefyamani,youth,youtu,youtub,ypg,ypre,yuan,yug,yugvani,yuki,yum,yumiko,yup,yuppi,yuuko,yuvi,yyc,yycfring,yycstorm,yycwalk,yycweath,yyeso,yyj,yzf,zaatari,zabadani,zacb,zach,zachlowenba,zachzaidman,zaibatsunew,zakbagan,zakuun,zaman,zamtriossu,zar,zarharzar,zarri,zayn,zaynmaiikist,zaynmalik,zeal,zehr,zenandemcfen,zenit,zergel,zero,zhejiang,zhenghxn,zicac,zimbabw,zimmer,zimmerman,zimpapersview,zionism,zionist,zip,ziphimup,zippednew,zipper,zippolin,ziuw,zmne,zodiac,zojadelin,zomatoau,zombi,zone,zonesthank,zoom,zotar,zouma,zourryart,zrnf,zumiez,zurich,zxatheti,åêfedex,åêi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [9]:

trainX , testX , trainY , testY = train_test_split( input_df , op_df )

lr = LogisticRegression()

trainX.fillna(0 , inplace = True )

/Users/Kunal/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [16]:
def update_word_count_new( x ):
    
    x['word_counts' ]['id' ] = x['id' ]
#     x['word_counts' ]['target' ] = x['target']
    return x['word_counts'] 
def preprocess_df( df ):
    df[ 'word_tokens' ] = df[ 'text' ].apply(clean_text_and_get_tokens).apply(lambda x : stem_and_lemmatize( ' '.join( x ) ) )
    
    df[ 'corpus_text'  ] = df[ 'word_tokens' ] .apply( lambda x : ' '.join( x  ) )
    
    new_df = get_tf_idf_df( df , 'corpus_text' )
    
    
    
    return new_df
    

In [11]:
from sklearn.model_selection import GridSearchCV

finder = GridSearchCV( lr , { 'C' : [ 0.2 , 0.4 , 0.6 , 0.8 ,1.0 , 1.5 , 2.0 , 2.5 , 3.0 ] } )

finder.fit(trainX , trainY)

best_clf = _

testX.fillna( 0 , inplace = True)

/Users/Kunal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Kunal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Kunal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Kunal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [12]:
predicted_vals = finder.predict( testX )

accuracy = accuracy_score( testY , predicted_vals )
accuracy

0.8082983193277311

In [13]:
best_clf = finder

In [14]:
imp_metrics = [ i for i in zip( input_cols , best_clf.best_estimator_.coef_[0] ) ]

imp_metrics.sort( key = lambda x : x[ 1 ] )

non_disaster_tweets_words = imp_metrics[:20]

non_disaster_tweets_words

[('love', -2.5034035966194366),
 ('crush', -2.3355472754554025),
 ('bag', -2.184930547227798),
 ('feel', -2.142991019775495),
 ('lol', -2.0612410954809506),
 ('let', -2.0227463120072504),
 ('fuck', -2.006270732295498),
 ('better', -1.9927846104311355),
 ('song', -1.9290294185243861),
 ('youtub', -1.8590838414108268),
 ('obliter', -1.85000219303062),
 ('blew', -1.8341948769949767),
 ('new', -1.8285303852715218),
 ('ebay', -1.77267252925717),
 ('lava', -1.7696125740643591),
 ('cri', -1.746119873423456),
 ('armageddon', -1.7174953307456402),
 ('tri', -1.7029175140345683),
 ('make', -1.6893929671581727),
 ('check', -1.6706425770706075)]

In [15]:
disaster_words = imp_metrics[-20:]

disaster_words

[('massacr', 2.4961216067616645),
 ('polic', 2.51700790247095),
 ('tornado', 2.574149273632029),
 ('debri', 2.6290670170307333),
 ('evacu', 2.6886640300995444),
 ('report', 2.7910607160698784),
 ('near', 2.8176029016257065),
 ('typhoon', 2.8279651220208506),
 ('flood', 2.830442537510205),
 ('storm', 2.858960634495234),
 ('murder', 2.89175059662601),
 ('earthquak', 3.0493093166623813),
 ('train', 3.157172942603945),
 ('california', 3.198006443643706),
 ('kill', 3.3571018248147904),
 ('fire', 3.4007587682423464),
 ('wildfir', 3.591110519525073),
 ('suicid', 3.781358982757379),
 ('hiroshima', 4.648159538387483),
 ('http', 4.836828738781723)]

In [17]:
len([ col for col in input_cols if len( col ) > 2 ] )

13129

In [18]:

len( input_cols )

13129

In [19]:
required_cols = get_meaningful_cols( input_cols)
len(required_cols)
# from nltk.stem import PorterStemmer

13129

In [20]:
required_cols

['aaceorg',
 'aan',
 'aar',
 'aaronthefm',
 'aashiqui',
 'aba',
 'abandon',
 'abandonedp',
 'abbandon',
 'abbott',
 'abbruchsimul',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcnew',
 'abcnorio',
 'abe',
 'aberdeen',
 'aberdeenfanpag',
 'aberdeenfc',
 'aberystwythshrewsburi',
 'abha',
 'abia',
 'abil',
 'abject',
 'abl',
 'ablaz',
 'ableg',
 'abninfvet',
 'aboard',
 'abomb',
 'abomin',
 'abort',
 'about',
 'abrancaballero',
 'absenc',
 'absolut',
 'abstorm',
 'abstract',
 'absurd',
 'absurdli',
 'abus',
 'abuseddesol',
 'abysmaljoin',
 'acaciapenn',
 'academia',
 'acarewornheart',
 'acc',
 'accept',
 'access',
 'accid',
 'accident',
 'accidentalpropheci',
 'accionempresa',
 'accompani',
 'accord',
 'accordingli',
 'account',
 'accuraci',
 'accus',
 'accustom',
 'acdelco',
 'ace',
 'acebab',
 'acebreakingnew',
 'acenewsdesk',
 'ach',
 'achedin',
 'achiev',
 'achimota',
 'acid',
 'acmilan',
 'acn',
 'acoust',
 'acousticmaloley',
 'acquiesc',
 'acquir',
 'acquisit',
 'acr',
 'acronym',
 '

In [ ]:
PorterStemmer().stem('likes')

In [21]:
test_df = pd.read_csv( '~/Downloads/nlp-getting-started/test.csv')

test_df = preprocess_df( test_df )

lr = best_clf.best_estimator_

lr.fit( input_df[input_cols].fillna( 0 ) , op_df )

cols = set( input_cols ) - set( test_df.columns )

for col in cols :

    test_df[ col ] = 0


/Users/Kunal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
l

In [22]:
predictions = lr.predict( test_df[ input_cols ] .fillna( 0 ) )

orig_test_df = pd.read_csv( '~/Downloads/nlp-getting-started/test.csv')

orig_test_df = orig_test_df[ ['id' ] ]

orig_test_df[ 'target' ] = predictions

orig_test_df.to_csv( 'submission_nlp.csv' , index = False )

In [ ]:
set(test_df.columns ) - set( input_cols )

In [ ]:
test_df_ = pd.read_csv( '~/Downloads/nlp-getting-started/test.csv')

In [ ]:
test_df_.head()

In [ ]:

corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
X

In [ ]:
X.toarray() 

In [ ]:
vectorizer.get_feature_names()


In [ ]:
pd.DataFrame(X.toarray() , columns = vectorizer.get_feature_names() )